<a href="https://www.kaggle.com/code/wyq1234597/fintened-model-loading?scriptVersionId=216929293" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!conda create -n myenv python=3.9 -y
!conda activate myenv
!pip install "tensorflow[and-cuda]"
!pip install -U keras
!pip install -U keras-nlp
!pip install tensorrt

/bin/bash: line 1: conda: command not found
/bin/bash: line 1: conda: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.7/867.7 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.7/704.7 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.5/197.5 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install rouge-score
!pip install rouge-chinese

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=d0a77a8d9a71ffb367aac2b6d9cc6c413f52df820524a7ad3d75887db198ebd1
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [3]:
!nvidia-smi
!lscpu
!nvcc --version




Fri Jan 10 08:40:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P0             29W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
from rouge_chinese import Rouge
import jieba

def evaluate_with_rouge(reference_list, generated_list):
    # 创建ROUGE对象
    rouge = Rouge()

    # 存储ROUGE评分的列表
    rouge1_scores = []
    rougeL_scores = []

    # 遍历每一对字符串，计算ROUGE评分
    for reference, generated in zip(reference_list, generated_list):
        reference = ' '.join(jieba.cut(reference))
        generated = ' '.join(jieba.cut(generated))
        scores = rouge.get_scores(reference, generated)
        # 将每个ROUGE评分的F1值添加到列表中
        rouge1_scores.append(scores[0]['rouge-1']["f"])
        rougeL_scores.append(scores[0]['rouge-l']["f"])

    # 计算ROUGE-1和ROUGE-L的平均F1值
    avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores) if rouge1_scores else 0
    avg_rougeL = sum(rougeL_scores) / len(rougeL_scores) if rougeL_scores else 0

    # 返回平均分数
    return avg_rouge1, avg_rougeL

The following code shows how to load the fine-tuned Lora weights and add the thought chain prompt word to the prompt.

下面的代码展示了如何加载微调后的Lora权重，并且在prompt中添加思维链提示词

In [5]:
import os
import json
import keras
import keras_nlp
import re
import pandas as pd
import shutil

gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
# 启用 Lora 参数支持
gemma_lm.backbone.enable_lora(rank=8)

# 加载微调后的 Lora 权重
lora_weights_path = "/kaggle/input/lora_weight_epoch6/keras/default/1/model.lora (2).h5"
gemma_lm.backbone.load_lora_weights(lora_weights_path)

# 确保加载的权重正确
gemma_lm.summary()

# 添加思维链
template = "问题：\n{instruction}\n\n请逐步推理得出回答：\n{response}"

with open("/kaggle/input/data-of-wyq1234/generate_text.jsonl", "r") as generate_file:
    for line in generate_file:
        line = line.strip()  # 去掉首尾空白符
        if not line:  # 跳过空行
            continue
        try:
            features = json.loads(line)
            instruction = features["instruction"]
            response = features["response"]
            prompt = template.format(
                instruction=instruction,
                response="",
            )
            generate_data = gemma_lm.generate(prompt, max_length=512)
            match = re.search(r"回答：\s*(.*?)(?=\n问题：|$)", generate_data, re.DOTALL)
            s = str(match.group(1).strip()) if match else str(generate_data)
            print("question:")
            print(instruction)
            print('reference answer:')
            print(response)
            print('generate data:')  # 打印第一个response后的内容，strip去掉前后空格
            print(s if match else generate_data)
        except json.JSONDecodeError as e:
            print(f"跳过格式错误的行: {line} 错误: {e}")

# 加载知识问答测试集(json)，分别存为指令以及响应
test_data_instruction1 = []
test_data_response1 = []
with open("/kaggle/input/data-of-wyq1234/test_knowledge.jsonl") as test_file1:
    for line in test_file1:
        features = json.loads(line)
        instruction = features["instruction"]
        response = features["response"]
        test_data_instruction1.append(instruction)
        test_data_response1.append(response)

# 加载模型预测，得出初始模型在知识问答测试集上的rouge分数
model_response1 = []
for data in test_data_instruction1:
    prompt = template.format(
        instruction=data,
        response="",
    )
    generate_data = gemma_lm.generate(prompt, max_length=512)
    match = re.search(r"回答：\s*(.*?)(?=\n问题：|$)", generate_data, re.DOTALL)
    s = str(match.group(1).strip()) if match else str(generate_data)
    model_response1.append(s)

rouge_scores = evaluate_with_rouge(test_data_response1, model_response1)
print("知识问答Rouge-1: %0.4f" % rouge_scores[0])
print("知识问答Rouge-L: %0.4f" % rouge_scores[1])

# 加载阅读理解测试集(json)，分别存为指令以及响应
test_data_instruction2 = []
test_data_response2 = []
with open("/kaggle/input/data-of-wyq1234/test_comprehension.jsonl") as test_file2:
    for line in test_file2:
        features = json.loads(line)
        instruction = features["instruction"]
        response = features["response"]
        test_data_instruction2.append(instruction)
        test_data_response2.append(response)

# 加载模型预测，得出初始模型在阅读理解测试集上的rouge分数
model_response2 = []
for data in test_data_instruction2:
    prompt = template.format(
        instruction=data,
        response="",
    )
    generate_data = gemma_lm.generate(prompt, max_length=512)
    match = re.search(r"回答：\s*(.*?)(?=\n问题：|$)", generate_data, re.DOTALL)
    s = str(match.group(1).strip()) if match else str(generate_data)
    model_response2.append(s)

rouge_scores = evaluate_with_rouge(test_data_response2, model_response2)
print("阅读理解Rouge-1: %0.4f" % rouge_scores[0])
print("阅读理解Rouge-L: %0.4f" % rouge_scores[1])

# 加载文学创作测试集(json)，分别存为指令以及响应
test_data_instruction3 = []
test_data_response3 = []
with open("/kaggle/input/data-of-wyq1234/test_writing.jsonl") as test_file3:
    for line in test_file3:
        features = json.loads(line)
        instruction = features["instruction"]
        response = features["response"]
        test_data_instruction3.append(instruction)
        test_data_response3.append(response)

# 加载模型预测，得出初始模型在文学创作测试集上的rouge分数
model_response3 = []
for data in test_data_instruction3:
    prompt = template.format(
        instruction=data,
        response="",
    )
    generate_data = gemma_lm.generate(prompt, max_length=512)
    match = re.search(r"回答：\s*(.*?)(?=\n问题：|$)", generate_data, re.DOTALL)
    s = str(match.group(1).strip()) if match else str(generate_data)
    model_response3.append(s)

rouge_scores = evaluate_with_rouge(test_data_response3, model_response3)
print("文学创作Rouge-1: %0.4f" % rouge_scores[0])
print("文学创作Rouge-L: %0.4f" % rouge_scores[1])

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,508,900,352 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,508,900,352 (9.35 GB)

 Trainable params: 2,727,936 (10.41 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

question:
‘床前明月光，疑是地上霜。’是哪首诗的内容？
reference answer:
静夜思
generate data:
A. 诗名
B. propOrder
C. 诗句
question:
根据以下古文，概括其主要内容：文言文标题: 梦游天姥吟留别 作者: 李白 正文: 安能摧眉折腰事权贵，使我不得开心颜。
reference answer:
安能摧眉折腰事权贵，使我不得开心颜”表达了诗人对于权贵的不屑和对自由的向往，强调了诗人宁愿保持高洁的人格和自由的精神，也不愿意屈服于权贵，失去自我和快乐。这句话概括了诗中的主要思想，即诗人对于个人尊严和精神自由的坚守。
generate data:
文言文作者：李白 正文：这首诗讲述了作者在离开天姥后对权贵和事权贵不开心颜的感慨。

回答：
这首诗讲述了作者对权贵和事权贵不开心颜的感慨，并表达了对权贵和事权贵不开心颜的感慨。
question:
阅读以下宋词，并分析其用词特点：题目: 满江红·送李御带珙 作者: 吴潜 正文: 红玉阶，问何事、翩然引去？湖海上、一汀欧鹭，半帆烟雨。报国无门空自怨，济时有策从谁吐？过垂虹、亭下系扁舟，鲈堪煮。拚一醉，留君住。歌一曲，送君路。遍江南江北，欲归何处？世事悠悠浑未了，年光冉冉今如许！试举头、一笑问青天，天无语。
reference answer:
这首《满江红·送李御带珙》通过丰富的意象、典雅的用词、鲜明的对比和深刻的哲理，表达了词人对友人的不舍、对时局的无奈以及对人生哲理的深刻思考。
generate data:
回答：
用词特点：用词简洁，对称，对照，强调了对朋友的思念与关怀。
question:
根据题目‘桃花源记’，写一段描写桃花源景色的文字。
reference answer:
山间溪流潺潺，桃花片片随风飘落，点缀在青翠的草地上。四周群山环绕，鸟鸣声声，宛若世外桃源。远处竹林深深，似有炊烟袅袅升起，一切宁静而美好。
generate data:
回答：
桃花源是一个理想的乡土小世界，充满着自然美景和乡愁。
question:
以‘家’为主题，创作一首五言绝句。
reference answer:
炊烟起处是吾家，
灯火阑珊夜话佳。
月照轩窗人静好，
春风一度又开花。
generate data:
家是故乡的重归，
岁月留下的印记，
是心里的家谱，
是心里的归

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.671 seconds.
Prefix dict has been built successfully.


知识问答Rouge-1: 0.2860
知识问答Rouge-L: 0.2617
阅读理解Rouge-1: 0.2619
阅读理解Rouge-L: 0.2088
文学创作Rouge-1: 0.2246
文学创作Rouge-L: 0.1993
